In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
cd /gdrive/My\ Drive/nn 

/gdrive/My Drive/nn


In [0]:
%tensorflow_version 1.x
import argparse
import os
import pickle
import sys
from itertools import combinations_with_replacement
import numpy as np

np.random.seed(7)  # for reproducibility

import tensorflow as tf
tf.random.set_random_seed(5005)

from sklearn.model_selection import train_test_split, KFold

# tf.python.control_flow_ops = tf


from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
from tensorflow.python.keras.layers.convolutional import Conv1D
from tensorflow.python.keras.layers.pooling import MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow.python.keras.backend as K

import matplotlib as mpl

mpl.use('Agg')
# from keras.utils.layer_utils import print_layer_shapes

import utils
sys.path.append(".")
from utils import load_data_merged
from utils import *

from tensorflow.python.keras.backend import set_session

model_path = "./models/final.h5"

CLASS_PARTIAL_LOSS = 0
CLASS_COMPLETELY_LOSS = 1 



TensorFlow 1.x selected.


In [0]:
def get_prediction(data):

  sess = tf.Session()
  tf.random.set_random_seed(5005)
  K.set_session(sess)

  session = K.get_session()
  init = tf.global_variables_initializer()
  session.run(init)

  with session.as_default():
      with session:

        model = load_model(model_path, custom_objects={'recall_TP': recall_TP,'recall_TN': recall_TN })
        pred = model.predict(data)

  return pred

In [0]:
def get_all_permutations(length):

  permutions1 = combinations_with_replacement(["C","G","A","T"], length)

  seq_l = []
  for i in permutions1:
    permutions2 = combinations_with_replacement(["C","G","A","T"], length)
    for j in permutions2:
      seq = "".join(i) + "CG" + "".join(j)

      if seq.count("CG") == 1:
        seq_l.append(seq)

  return list(set(seq_l))

In [0]:
def seq_to_mat(seq):
    seq_len = len(seq)
    seq = seq.replace('A', '0')
    seq = seq.replace('a', '0')
    seq = seq.replace('C', '1')
    seq = seq.replace('c', '1')
    seq = seq.replace('G', '2')
    seq = seq.replace('g', '2')
    seq = seq.replace('T', '3')
    seq = seq.replace('t', '3')
    seq = seq.replace('U', '3')
    seq = seq.replace('u', '3')
    seq = seq.replace('N', '4')  # some cases have N in sequence
    seq = seq.replace('n', '4')
    seq_code = np.zeros((4, seq_len), dtype='float16')

    for i in range(seq_len):
        if seq[i] == "_":
          continue
        
        elif int(seq[i]) != 4:
            seq_code[int(seq[i]), i] = 1

        else:
            seq_code[0:4, i] = np.tile(0.25, 4)

    return np.transpose(seq_code)

In [0]:
def predict_for_seq_length(length):
  sequences = get_all_permutations(length)
  seq_length = len(sequences[0])
  filler = int((150-seq_length) /2 )
  full_sequences = []
  for seq in sequences:
    full_seq = filler * "_" + seq + filler * "_"
    full_sequences.append(full_seq)

  seq_mat = np.array([seq_to_mat(seq) for seq in full_sequences])
  pred = get_prediction(seq_mat)
  output = [(sequences[i], pred[i][0] * 100) for i in range(pred.shape[0])]
  return output

In [0]:
output = predict_for_seq_length(3)

In [0]:
with open("./3.csv", "w") as c:
  c.writelines(["%s,%s\n" %(i[0], i[1]) for i in output])